<br>
<img style="float: left"; src="img\bip.jpeg" width="60">
<img style="float: left; margin-left: 30px;"; src="img\crif-customer-logo.png" width="100">
<br>
<br>
<br>
<br>

# SQL - Standard Query Language

<br>
<img style="float: center"; src="img\sql.png" width="300">
<br>

SQL is a formal language for communicating with a database to articulate precisely what information you want from a collection of database tables.

Databases are *structured* repositories for data in that database designers must be specific about defining a common set of data items that will be available for each record in a table. This includes defining each column name and data type.

Python has a built-in tools for accessing a database and sending or receiving data. In this notebook, we will explore these tools.

In [1]:
%load_ext sql
%sql sqlite:///testdb.sqlite

'Connected: @testdb.sqlite'

# Content:
[1. Creating a Table](#1)  <br>
[2. Schemas](#2)  <br>
[3. Selecting data](#3)  <br>
[4. Case-When](#4)  <br>
[5. Groupby](#5)  <br>
[6. Combining tables](#6)  <br>
[7. Union](#7)  <br>
[8. Creating SQL Tables with Pandas](#8)  <br>

# SQL Syntax in Detail

**Key nouns:**
    - "Relational" database
    - Table
    - Row
    - Column
    - (Primary key)

**Key verbs:**
    - `CREATE TABLE` / `DROP TABLE` to modify the structure of the database
    - `SELECT` to read out some rows
    - `INSERT` / `UPDATE` / `DELETE` to modify (or delete) some rows of tables

**Key modifiers:**
    - `WHERE`: impose condition
    - `ORDER BY`: sort
    - `GROUP BY`: aggregate (e.g. pivot tables in Excel)
    - `JOIN`: combining tables in one query

<a id='1'></a>
# 1. Creating a Table

You create a table using the syntax

```SQL
CREATE TABLE tb ([schema])
```

We can insert rows by using the syntax

```SQL
INSERT INTO tb ([column]) VALUES ([values]);
```

**Gotchas:**
 - For strings you 'must' use single quotes.  (SQLite and MySQL don't care.  Postgres does.)
 - The primary key is required to uniquely specify a row.  If you do not specify it when you insert an item (and you usually don't) a new unique key will be assigned to you.  If you try to insert a primary key value that already exists for the table, you will get an error.
 - While not necessarily required, it's a good idea to preceded table creation by table deletion:

```SQL
DROP TABLE IF EXISTS tb
```
 
Below are some examples: we'll worry about the details later.

In [2]:
%%sql
-- This table demonstrates how to create a table, insert values, and display results
    
DROP TABLE IF EXISTS Employees1;
CREATE TABLE Employees1 (
    employee_id INTEGER PRIMARY KEY,
    name TEXT,
    age FLOAT
);

INSERT INTO Employees1 (name, age) VALUES ('Bob', 2.);
INSERT INTO Employees1 (age, name) VALUES (1., 'John');
INSERT INTO Employees1 (employee_id, age, name) VALUES (4, 3., 'Jack');

SELECT * FROM Employees1;

 * sqlite:///testdb.sqlite
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


employee_id,name,age
1,Bob,2.0
2,John,1.0
4,Jack,3.0


<a id='2'></a>

# 2. Schemas

SQL is a typed language (if it can be called a language).  That means that its tables have a **schema** or a description of the structure of a database (tables, rows, etc.).  Each column of a table must specify its **type** and a few other attributes.  The basic types are:

- `INTEGER` (or `INT`): integer-valued numeric data
- `FLOAT`: non-integer numeric data
- `TEXT`: strings or textual data
- `DATE`: a date object

Columns can have optional attributes:

- `NOT NULL`: column values can always be null (not specified).  `NOT NULL` forces the table to be specified.
- `DEFAULT`: specify the default value in this column.

In [3]:
%%sql
-- This table demonstrates optional values
    
DROP TABLE IF EXISTS Employees1;
CREATE TABLE Employees1 (
    employee_id INTEGER PRIMARY KEY, 
    name TEXT NOT NULL,
    age FLOAT
);

INSERT INTO Employees1 (name) VALUES ('Bob');
INSERT INTO Employees1 (name, age) VALUES ('Jack', 3.14);

SELECT * FROM Employees1;

 * sqlite:///testdb.sqlite
Done.
Done.
1 rows affected.
1 rows affected.
Done.


employee_id,name,age
1,Bob,None
2,Jack,3.14


## Multiple Tables and the Relational Data Model

The database consists of:
 - **Tables**: Employees, Tasks, and Assignments.  
      > Each table represents a different kind of "relation" between bits of data.
    
 - Each of the tables has many **rows** (sometimes called "relations").  For instance, each row of the `Employees` table represents information about a single employee:  
 
      > Elements of a single column are _homogeneous_ (of the same kind, or data type) but apply to _different logical entities_.
      
 - The information about a single employee is spread amongst the **columns** of the table:
 
      > Elements of a single row are _heterogeneous_ but apply to _a single logical entity_.
      
 - It is good practice to be able to specify each row of each table by a **primary key**: that is by some column, or small combination of columns, that uniquely specifies the row.  The columns `employee_id`, `task_id`, and `assignment_id` are all of this type.  This way, if you want to refer to this row in another table you can do it succinctly by this primary key.  SQL enforces uniqueness of the primary key.


In [4]:
%%sql
DROP TABLE IF EXISTS Employees;
CREATE TABLE Employees (
    employee_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    hire_date DATE NOT NULL,
    title TEXT NOT NULL,
    phone_number TEXT NOT NULL
);

INSERT INTO Employees VALUES (1, 'Jenny Smith', '2010-12-1', 'Chief Music Office', '867-5309');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('CEO', 'John Doe', '555-1234', '2011-12-1');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 1', '555-1201', '2012-12-01');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 2', '555-1202', '2012-12-03');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 3', '555-1203', '2012-12-05');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 4', '555-1204', '2012-12-07');

SELECT * FROM Employees;

 * sqlite:///testdb.sqlite
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


employee_id,name,hire_date,title,phone_number
1,Jenny Smith,2010-12-1,Chief Music Office,867-5309
2,John Doe,2011-12-1,CEO,555-1234
3,Worker Bee 1,2012-12-01,Worker,555-1201
4,Worker Bee 2,2012-12-03,Worker,555-1202
5,Worker Bee 3,2012-12-05,Worker,555-1203
6,Worker Bee 4,2012-12-07,Worker,555-1204


In [5]:
%%sql
DROP TABLE IF EXISTS Tasks;
CREATE TABLE Tasks (
    task_id INTEGER PRIMARY KEY,
    deadline DATE NOT NULL,
    status INT NOT NULL DEFAULT 0,
    description TEXT
);

INSERT INTO Tasks (deadline, description, status) VALUES ('2012-12-11', 'Hire worker bees.', 1);
INSERT INTO Tasks (deadline, description) VALUES ('2014-07-18', 'Save the world.');
INSERT INTO Tasks (deadline, description, status) VALUES ('2014-12-11', 'Cost cutting on worker bees.', 0);

SELECT * FROM Tasks;

 * sqlite:///testdb.sqlite
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


task_id,deadline,status,description
1,2012-12-11,1,Hire worker bees.
2,2014-07-18,0,Save the world.
3,2014-12-11,0,Cost cutting on worker bees.


In [6]:
%%sql
DROP TABLE IF EXISTS Assignments;
CREATE TABLE Assignments (
    assignment_id INTEGER PRIMARY KEY,
    task_id INTEGER NOT NULL,
    employee_id INTEGER NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    charge_code TEXT NOT NULL,  -- This should probably be its own table!
    fraction_assignment REAL NOT NULL
);

INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (1, 2, '2012-01-01', '2012-12-31', 0.5, 'INTERNAL');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 3, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 4, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 5, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 6, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 2, '2013-01-01', '2014-07-18', 0.75, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (3, 2, '2013-01-01', '2014-07-18', 0.25, 'INTERNAL');

SELECT * FROM Assignments;

 * sqlite:///testdb.sqlite
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


assignment_id,task_id,employee_id,start_date,end_date,charge_code,fraction_assignment
1,1,2,2012-01-01,2012-12-31,INTERNAL,0.5
2,2,3,2013-01-01,2014-07-18,CJX5,1.0
3,2,4,2013-01-01,2014-07-18,CJX5,1.0
4,2,5,2013-01-01,2014-07-18,CJX5,1.0
5,2,6,2013-01-01,2014-07-18,CJX5,1.0
6,2,2,2013-01-01,2014-07-18,CJX5,0.75
7,3,2,2013-01-01,2014-07-18,INTERNAL,0.25


In [7]:
%%sql

DROP TABLE IF EXISTS Pension_Choices;
CREATE TABLE Pension_Choices (
    pension_id INTEGER PRIMARY KEY,
    employee_id INTEGER NOT NULL, 
    plan_name TEXT NOT NULL,
    funding_amount INTEGER NOT NULL);

INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (3, 'Retirement Plan A', '3000');
INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (4, 'Retirement Plan A', '5000');
INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (5, 'Retirement Plan B', '5000');
INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (6, 'Retirement Plan B', '1000');

SELECT * FROM Pension_Choices;

 * sqlite:///testdb.sqlite
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


pension_id,employee_id,plan_name,funding_amount
1,3,Retirement Plan A,3000
2,4,Retirement Plan A,5000
3,5,Retirement Plan B,5000
4,6,Retirement Plan B,1000


<a id='3'></a>

# 3. Selecting data

The basic way to read rows from a single table is

```SQL
SELECT expression1, expression2, ... FROM table_name [WHERE condition] [GROUP BY columns] [ORDER BY columns] [LIMIT number];
```

Brackets denote parts which are optional.  We'll explain everything except for `GROUP BY`.  Let us explain the parts:
   1. __"Expressions": __
   
   The basic example is just a column name.  Another example is the placeholder `*`: it indicates _all_ columns.

   1. __`WHERE` (condition):__
   
   An example would be `column_name = value`, and for partial string matches there is a special operator `LIKE`.  You can combine these with the usual binary operators (AND, OR) and parentheses for grouping. 

   1. __`ORDER BY` (sorting):__
   
   You can add a `ORDER BY column_name1, column_name2, ...` to sort the results by the given column (applied in left-to-right order).  By default the sorting is ascending.  If you want it to be descending write e.g. `column_name1 DESC` (you can also write `ASC` for ascending to disambiguate).
 
   1. __`LIMIT`:__
   
   If you want to just get at most the first N rows that would be returned, add on `LIMIT N`.
   
   1. __`AS` (named expressions):__
   
   You can specify new names for the expressions that you're selecting.  This is especially useful when you're using aggregate functions: 
```SQL
SELECT SUM(salary) as 'Total Salary' FROM ...
```

Now for a bunch of examples:

In [8]:
%%sql

SELECT
    count(*) as Count
FROM
    Tasks
WHERE
    status=0;

 * sqlite:///testdb.sqlite
Done.


Count
2


In [9]:
%%sql

SELECT
    assignment_id,
    julianday(end_date) - julianday(start_date) AS duration
FROM
    Assignments
WHERE
    employee_id=2
ORDER BY 
    duration DESC;

 * sqlite:///testdb.sqlite
Done.


assignment_id,duration
6,563.0
7,563.0
1,365.0


<a id='4'></a>

# 4. CASE - WHEN

Another useful structure that can be an alternative to `WHERE` statements is the `CASE WHEN` command. It essentially operates as a switch statement where you can specify multiple conditions and different outcomes for each condition.

In [10]:
%%sql

SELECT
    assignment_id, CASE
        WHEN julianday(end_date) - julianday(start_date) < 365 THEN "months"
        WHEN julianday(end_date) - julianday(start_date) > 365 THEN "years"
        ELSE "one year"
        END "length"
FROM Assignments;

 * sqlite:///testdb.sqlite
Done.


assignment_id,length
1,one year
2,years
3,years
4,years
5,years
6,years
7,years


<a id='5'></a>

# 5. Groupby (aggregation)
   
Finally, add a `GROUP BY column_name1, ...` term to aggregate the results along the listed columns.   This means that for _other_ columns you mention in the expressions you must explicitly specify an _aggregate method_ (e.g. `SUM(column_1)`).
   
   You can use these aggregate methods even without a GROUP BY, to get information for the whole table: e.g.
  ```SQL 
SELECT COUNT(1), SUM(salary) FROM employees GROUP BY department;
   ```
   returns the number of employees and their total salary by department.  Below are some more examples

In [11]:
%%sql

SELECT plan_name, count(1),sum(funding_amount) 
FROM Pension_choices 
GROUP BY plan_name

 * sqlite:///testdb.sqlite
Done.


plan_name,count(1),sum(funding_amount)
Retirement Plan A,2,8000
Retirement Plan B,2,6000


In [12]:
%%sql

SELECT
    task_id,
    SUM((julianday(end_date) - julianday(start_date))*fraction_assignment) as Total_Worker_Days
FROM
    Assignments
GROUP BY task_id
ORDER BY Total_Worker_Days DESC;

 * sqlite:///testdb.sqlite
Done.


task_id,Total_Worker_Days
2,2674.25
1,182.5
3,140.75


<a id='6'></a>

# 6. Combining tables: subqueries and joins 

It's pretty boring to only use one table.  We can combine data from multiple tables in two main ways: subqueries and joins.

### Subqueries
Let's just see it by example:

In [13]:
%%sql

SELECT ad.task_id, ad.Total_Worker_Days 
    FROM (SELECT task_id, SUM((julianday(end_date) - julianday(start_date))*fraction_assignment) as Total_Worker_Days
            FROM Assignments
            GROUP BY task_id
            ORDER BY Total_Worker_Days DESC) as ad
    WHERE ad.task_id=(SELECT task_id 
                      FROM Tasks 
                      WHERE description LIKE 'Save%');

 * sqlite:///testdb.sqlite
Done.


task_id,Total_Worker_Days
2,2674.25


### Joins:

    
```SQL
SELECT * FROM table1, table2
        WHERE table1.column_name1 = table2.column_name2;
```

In SQL terms, this is called an `INNER JOIN`: You do not get a row in the result of the SELECT unless you can match a row in table 1 with a match in table 2 along the given columns.  

There is alternate syntax for it:
```SQL
SELECT * FROM table1
        INNER JOIN table2
        ON table1.column_name1=table2.column_name2;
```

In [14]:
%%sql

SELECT a.*, t.* 
  FROM Assignments a, Tasks t 
    WHERE a.task_id=t.task_id 
    AND employee_id=2 
    AND status=0
    ORDER BY t.deadline;

 * sqlite:///testdb.sqlite
Done.


assignment_id,task_id,employee_id,start_date,end_date,charge_code,fraction_assignment,task_id_1,deadline,status,description
6,2,2,2013-01-01,2014-07-18,CJX5,0.75,2,2014-07-18,0,Save the world.
7,3,2,2013-01-01,2014-07-18,INTERNAL,0.25,3,2014-12-11,0,Cost cutting on worker bees.


<a id='7'></a>

# 7. UNION

These commands are used for joining data row-wise either between tables or within the same table. In general, the syntax looks like `(SELECT statement) UNION (SELECT STATEMENT)` where both SELECT statements have the same columns.

In [15]:
%%sql

SELECT task_id, employee_id FROM Assignments WHERE employee_id = 2
UNION 
SELECT task_id, employee_id FROM Assignments WHERE employee_id = 4;

 * sqlite:///testdb.sqlite
Done.


task_id,employee_id
1,2
2,2
2,4
3,2


<a id='8'></a>

# 8. Creating SQL Tables with Pandas: A Shortcut

Pandas is awesome in many ways, and one of them is its ability to create database tables for you. If you have a DataFrame and a database connection, it will write the SQL CREATE statements to build the table schema and insert all of the rows. It works like this.

In [16]:
import pandas as pd
import sqlite3

In [17]:
# sample data
fha_by_tract = pd.read_csv('data/2013_Gaz_tracts_national.tsv', sep='\t')
fha_by_tract.columns = [c.strip() for c in fha_by_tract.columns] 

# create db connection
conn = sqlite3.connect('testdb.sqlite')

# create a database table with the proper types to match the DataFrame types and insert all of the rows.
fha_by_tract.to_sql('gaz_tracts', conn, if_exists='replace', index=False)

74001

In [18]:
fha_by_tract

,USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001020100,9809939,36312,3.788,0.014,32.481794,-86.490249
1,AL,1001020200,3340498,5846,1.290,0.002,32.475758,-86.472468
2,AL,1001020300,5349274,9054,2.065,0.003,32.474024,-86.459703
3,AL,1001020400,6382705,16244,2.464,0.006,32.471030,-86.444835
4,AL,1001020500,11397734,48412,4.401,0.019,32.458916,-86.421817
...,...,...,...,...,...,...,...,...
73996,PR,72153750501,1795740,0,0.693,0.000,18.031240,-66.867250
73997,PR,72153750502,689929,0,0.266,0.000,18.024746,-66.860442
73998,PR,72153750503,3322868,1952,1.283,0.001,18.023325,-66.874841
73999,PR,72153750601,10987037,4527,4.242,0.002,18.017809,-66.839070


That's it. If you have a DataFrame and like it, you can make a database table out of it.

Now we can verify the table exists and has data.

In [19]:
%%sql

SELECT * FROM gaz_tracts LIMIT 10;

 * sqlite:///testdb.sqlite
Done.


USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
AL,1001020100,9809939,36312,3.788,0.014,32.4817943,-86.4902488
AL,1001020200,3340498,5846,1.29,0.002,32.475758,-86.4724678
AL,1001020300,5349274,9054,2.065,0.003,32.4740243,-86.4597033
AL,1001020400,6382705,16244,2.464,0.006,32.4710304,-86.4448353
AL,1001020500,11397734,48412,4.401,0.019,32.4589157,-86.4218165
AL,1001020600,8020363,60048,3.097,0.023,32.4473674,-86.4768327
AL,1001020700,22414817,775005,8.654,0.299,32.4303487,-86.4369714
AL,1001020801,124278423,8117643,47.984,3.134,32.4180838,-86.527137
AL,1001020802,190810946,678013,73.673,0.262,32.5466429,-86.5312317
AL,1001020900,292756803,516150,113.034,0.199,32.6370123,-86.5149469


### Exercise

Return the `sum` of these columns `[AWATER_SQMI, ALAND, AWATER]` and the `average` value for these columns `[INTPTLAT, INTPTLONG]` for each `USPS` that satisfy these conditions: 
- starts with `A`
- have a `sum` of `AWATER_SQMI` below `1500`


*Hint: You have to use the statement `HAVING`*


*(10 min)*

In [39]:
%%sql

SELECT USPS, sum(AWATER_SQMI), sum(ALAND), sum(AWATER), Avg(INTPTLAT), Avg(INTPTLONG)
FROM gaz_tracts
WHERE USPS LIKE 'a%'
GROUP BY USPS
HAVING sum(AWATER_SQMI) < 1500;


 * sqlite:///testdb.sqlite
Done.


USPS,sum(AWATER_SQMI),sum(ALAND),sum(AWATER),Avg(INTPTLAT),Avg(INTPTLONG)
AR,1142.3999999999985,134772954601,2958815561,35.063891292274036,-92.53671764387744
AZ,396.8569999999992,294205037082,1027846143,33.350129589121885,-111.86668900786387
